In [26]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import pandas as pd


In [27]:
from bs4 import BeautifulSoup

In [171]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
import pymorphy2 # Морфологический анализатор.

In [65]:
# отключаем warnings
import warnings

warnings.filterwarnings('ignore')

In [30]:
# cleantext = BeautifulSoup(raw_html, "lxml").text

In [31]:
df_train_raw = pd.read_csv('data/train.csv', sep='\t')

In [32]:
df_train_raw.head()

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [33]:
df_train_raw.shape

(200000, 4)

In [34]:
df_test = pd.read_csv('data/test.csv', sep='\t')

In [35]:
df_test.head()

,id,name,description
0,200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...
1,200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...
2,200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...
4,200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...


In [36]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
    

In [37]:
def prepare_data(dataframe):
    df = dataframe.copy()
    # clearhtml
    df['description'] = df['description'].apply(lambda x : cleanhtml(x) )
    # replace &quot to "
    df['description'] = df['description'].str.replace('&quot;','"')
    df['description'] = df['description'].str.replace('\u200b','')
    # uni name and description
    df['text'] = df['name'] + ' ' + df['description'] 
    # drop
    df = df.drop(columns=['name', 'description', 'id'], axis=1)
    return df
    

In [151]:
def f_tokenizer(s):
    morph = pymorphy2.MorphAnalyzer()
    if type(s) == str:
        t = s.split(' ')
    else:
        t = s
    f = []
    for j in t:
        m = morph.parse(j.replace('.',''))
        if len(m) != 0:
            wrd = m[0]
            if wrd.tag.POS not in ('NUMR','PREP','CONJ','PRCL','INTJ'):
                f.append(wrd.normal_form)
    return f

In [156]:
N = 50000

In [157]:
%%time 
df_train = prepare_data(df_train_raw[:N])`

CPU times: user 3.64 s, sys: 357 ms, total: 4 s
Wall time: 14.6 s


In [158]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
target    50000 non-null int64
text      50000 non-null object
dtypes: int64(1), object(1)
memory usage: 781.3+ KB


In [159]:
df_tmp_test = prepare_data(df_test[:N])


In [160]:
df_tmp_test.head()

,text
0,Дизайнер-консультант мебели Обязанности: Рабо...
1,Продавец-консультант (ТЦ на Пушкина) Обязаннос...
2,Менеджер по продажам Торговый Дом «Форт» это в...
3,Продавец-консультант в магазин одежды (ТЦ Волн...
4,Специалист по охране труда Обязанности: осущ...


In [202]:
df_id = df_test[:N].drop(columns=['name', 'description'], axis=1)

In [203]:
df_id.head()

,id
0,200000
1,200001
2,200002
3,200003
4,200004


In [161]:
label = 'target'
RND_SEED = 123
idx_features = df_train.columns != label

In [162]:
X = df_train.loc[:, idx_features].values.tolist()
y = df_train.loc[:, ~idx_features].values
tests = df_tmp_test["text"].values

In [163]:
texts = []
for item in X:
    texts.append(item[0])

In [164]:
texts

['Заведующий отделом/секцией в магазин YORK (Уручье) В НОВЫЙ МАГАЗИН YORK (хозтовары) приглашаем на постоянную работу руководителя секции (хозтовары). Обязанности:     организация эффективного и культурного обслуживания покупателей организация приемки-сдачи товаров на склад консультация покупателей по вопросам, касающимся оказываемых услуг контроль своевременной подачи товаров в торговую секцию проверка качества, сроков годности ТМЦ, наличие маркировок, ценников на товарах организация бесперебойной работы товарной секции и участка склада, закрепленного за ней контроль за сохранностью товаров, торгового оборудования и прочих материальных ценностей инвентаризация товаров    Требования:  знание товарной группы знание правил приемки товара опыт работы в розничной торговле в качестве продавца 6 р. / заместителя заведующего секцией/отделом ответственность, лидерские качества, опыт управления коллективом      Условия:  НОВЫЙ современный магазин розничной торговли хозтоварами хороший коллектив

In [176]:
model = Pipeline([
        ('union', FeatureUnion(
                # Use FeatureUnion to combine the features from subject and body
            transformer_list=[
                    # Pipeline for standard bag-of-words model
                ('text', Pipeline([
                        ('vect', CountVectorizer(
                            tokenizer=None, 
                            analyzer="word",
                        )),
                        ('tfidf', TfidfTransformer(
                            norm=None, 
                            smooth_idf=False,
                        )),
                    ])
                ),
            ],# end  transformer_list
                # weight components in FeatureUnion
            transformer_weights={
                'text': 0.5,
            }) # end FeatureUnion
        ),# end union
        ('clf', MultinomialNB()),
]) # end pipeline

model_2 = Pipeline([
        ('union', FeatureUnion(
                # Use FeatureUnion to combine the features from subject and body
            transformer_list=[
                    # Pipeline for standard bag-of-words model
                ('text', Pipeline([
                        ('tfidf', TfidfVectorizer(
                            norm=None, 
                            smooth_idf=False,
                        )),
                    ])
                ),
            ],# end  transformer_list
                # weight components in FeatureUnion
            transformer_weights={
                'text': 0.5,
            }) # end FeatureUnion
        ),# end union
        ('clf', MultinomialNB()),
]) # end pipeline


In [177]:
%%time 
# Схема кросс-валидации
n_splits = 5
cv = StratifiedKFold(
        n_splits=n_splits, shuffle=True, 
        random_state=RND_SEED)

scores = cross_val_score(
    model_2, texts, y,
    scoring='roc_auc', 
    cv=cv, 
n_jobs=-1)

CPU times: user 826 ms, sys: 2.25 s, total: 3.08 s
Wall time: 43.3 s


In [178]:
scores.mean(), scores.std()

(0.9539907717011651, 0.0019164691798865317)

In [189]:
model_2.fit(texts, y)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_...former_weights={'text': 0.5})), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [193]:
predictions = model_2.predict(tests)

In [204]:
df_id['target'] = pd.DataFrame(predictions.tolist())

In [206]:
df_id.shape

(50000, 2)

In [207]:
df_id.to_csv(path_or_buf='./submission/submission.cvs', sep=',')